In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import regularizers


#### Load Dataset
load the train and test dataset from your local directory

In [ ]:
# Load the train and test datasets
train_folder = 'train'
test_folder = 'test'

X_train = pd.read_csv('AirQuality_X_train.csv')
y_train = pd.read_csv('AirQuality_Y_train.csv')
X_test = pd.read_csv('AirQuality_X_test.csv')
y_test = pd.read_csv('AirQuality_Y_test.csv')
print(y_test)

In [ ]:
# Convert DataFrames to NumPy arrays for slicing
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values
print(y_test)

### Scaling the Data
Here we scale the data using MinMaxScaler so that all values are between 0 and 1, which is critical for improving the performance of neural networks.

In [ ]:
# Scaling the features and targets using MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)


### Defining Time Steps and Reshaping Data
The time steps define how much historical data will be used to predict the future. Here, we use the previous 60 hours of data to predict the next 10 hours.

In [ ]:
# Define the number of time steps and sequence length for input/output
n_steps = 60  # 60 hours for input
n_outputs = 10  # Predict the next 10 hours
n_features = X_train.shape[1]
print(n_features)

In [ ]:
# Reshape data to 3D format for RNN/LSTM input
def create_sequences(X, Y, n_steps, n_outputs):
    Xs, Ys = [], []
    for i in range(len(X) - n_steps - n_outputs):
        Xs.append(X[i:(i + n_steps), :])
        Ys.append(Y[(i + n_steps):(i + n_steps + n_outputs), :])
    return np.array(Xs), np.array(Ys)

X_train_seq, y_train_seq = create_sequences(X_train, y_train, n_steps, n_outputs)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, n_steps, n_outputs)
print(f"X_train_seq shape: {X_train_seq.shape}")
print(f"y_train_seq shape: {y_train_seq.shape}")

### Building the Model

In [ ]:

#Update the model here



#### Compiling and Training the Model
The model is compiled using the Adam optimizer and mean squared error (MSE) as the loss function. We use early stopping and learning rate scheduling to optimize training.

In [ ]:
# Compile the model with a lower learning rate
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse')

# Early stopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_schedule = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the model with early stopping

history = model.fit(X_train_seq, y_train_seq, epochs=100, batch_size=64, validation_split=0.2,
                    callbacks=[early_stopping, lr_schedule])

#### Plotting Training and Validation Loss
Visualize the training and validation loss over time to ensure the model is learning and not overfitting.

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.grid(True)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

#### Evaluating the Model
Evaluate the performance of the model on the test set using MSE.

In [ ]:
# Evaluate the model on the test set
test_loss = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss (MSE): {test_loss}")

#### Making Predictions and Plotting Results
Here we visualize the true values versus predicted values for CO, NOx, and Benzene.

In [ ]:
# Forecast the next 10 time steps for a sample test sequence
y_pred = model.predict(X_test_seq[:1])
print(y_pred)

In [ ]:
# Plot true vs predicted values for CO
plt.plot(range(n_steps), X_test_seq[0, :, 0], label='Input Sequence (CO Sensor)')
plt.plot(range(n_steps, n_steps + n_outputs), y_test_seq[0, :, 0], 'ro-', label='True CO')
plt.plot(range(n_steps, n_steps + n_outputs), y_pred[0, :, 0], 'bx-', label='Predicted CO')
plt.legend()
plt.grid(True)
plt.xlabel("Time Steps")
plt.ylabel("CO Concentration")
plt.show()

In [ ]:
# Plot true vs predicted values for NOx
plt.plot(range(n_steps), X_test_seq[0, :, 2], label='Input Sequence (NOx Sensor)')
plt.plot(range(n_steps, n_steps + n_outputs), y_test_seq[0, :, 2], 'ro-', label='True NOx')
plt.plot(range(n_steps, n_steps + n_outputs), y_pred[0, :, 2], 'bx-', label='Predicted NOx')
plt.legend()
plt.grid(True)
plt.xlabel("Time Steps")
plt.ylabel("NOx Concentration")
plt.show()

In [ ]:
#  # Plot true vs predicted values for Benzene (C6H6)
plt.plot(range(n_steps), X_test_seq[0, :, 1], label='Input Sequence (Benzene Sensor)')
plt.plot(range(n_steps, n_steps + n_outputs), y_test_seq[0, :, 1], 'ro-', label='True Benzene')
plt.plot(range(n_steps, n_steps + n_outputs), y_pred[0, :, 1], 'bx-', label='Predicted Benzene')
plt.legend()
plt.grid(True)
plt.xlabel("Time Steps")
plt.ylabel("Benzene Concentration")
plt.show()

#### Calculate MSE for Each Pollutant
Finally, we evaluate the Mean Squared Error (MSE) for each pollutant separately.

In [ ]:
# Evaluate MSE for each pollutant individually
from sklearn.metrics import mean_squared_error

In [ ]:
y_pred_full = model.predict(X_test_seq)
mse_co = mean_squared_error(y_test_seq[:, :, 0], y_pred_full[:, :, 0])
mse_benzene = mean_squared_error(y_test_seq[:, :, 1], y_pred_full[:, :, 1])
mse_nox = mean_squared_error(y_test_seq[:, :, 2], y_pred_full[:, :, 2])

In [ ]:
print(f"Test MSE for CO: {mse_co}")
print(f"Test MSE for Benzene: {mse_benzene}")
print(f"Test MSE for NOx: {mse_nox}")